In [ ]:
import torch
from torch import nn

class BatchNorm(nn.Module):
    def __init__(self, nf, mom=0.1, eps=1e-5):
        super().__init__(())
        self.mom = mom
        self.eps = eps
        self.mult = nn.Parameter(1.)
        self.add = nn.Parameter(0.)
        self.register_buffer('vars', torch.ones(1,nf,1,1))
        self.register_buffer('mean', torch.zeros(1,nf,1,1))

    def update_stats(self, x):
        m = x.mean((0,2,3), keepdim=True)
        v = x.var((0,2,3), keepdim=True)
        with torch.no_grad():
            self.mean.lerp_(m, self.mom)
            self.vars.lerp_(v, self.mom)
        return m, v

    def forward(self, x):
        if self.training:
            with torch.no_grad(): m, v = self.update_stats(x)
        else: m, v = self.mean, self.vars
        x = (x - m) / torch.sqrt(v + + self.eps)
        return x * self.mult + self.add